<a href="https://colab.research.google.com/github/MazenMarei25/Machine-Learning-Practice-/blob/main/Computer%20Vision/TL_Car_parts_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gpiosenka_car_parts_40_classes_path = kagglehub.dataset_download('gpiosenka/car-parts-40-classes')

print('Data source import complete.')


Importing Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/car-parts-40-classes")

print("Path to dataset files:", path)

In [ ]:
train_dir = "/kaggle/input/car-parts-40-classes/car parts/train"
test_dir = "/kaggle/input/car-parts-40-classes/car parts/test"
valid_dir = "/kaggle/input/car-parts-40-classes/car parts/valid"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
import time
import os

In [ ]:
# Naming the model with a timestamp to save it for later
ckpt_dir = 'kaggle/working/CV/checkpoints'

NAME = "car-parts-cnn-classifier-{}".format(int(time.time()))

os.makedirs(ckpt_dir, exist_ok=True)

ckpt_path = os.path.join(
    ckpt_dir,
    f"{NAME}_epoch-{{epoch:02d}}.keras"
)

checkpoint = ModelCheckpoint(
    filepath=ckpt_path,
    save_weights_only=False,   # saves full model
    save_freq='epoch'          # saves after every epoch
)


Checking the dataset

In [ ]:
# Create from the train,test,val folders the datasets in the format expected by keras
# tf.keras.preprocessing.image_dataset_from_directory() automatically creates an interable of batches of image label pairs

IMAGE_SIZE = (224,224)
BATCH_SIZE = 8

train_ds=tf.keras.preprocessing.image_dataset_from_directory(train_dir,label_mode="int",batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,shuffle="True",seed=42,color_mode="rgb")

test_ds=tf.keras.preprocessing.image_dataset_from_directory(test_dir,label_mode="int",batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,shuffle="True",seed=42,color_mode="rgb")

valid_ds=tf.keras.preprocessing.image_dataset_from_directory(valid_dir,label_mode="int",batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,shuffle="True",seed=42,color_mode="rgb")

In [ ]:
# take 1 batch of the image and show its size
for images, labels in train_ds.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

    # (32, 256, 256, 3) | 32 batches each image is 160x160 and has 3 channels RGB

In [ ]:
# print the class names just to make sure
class_names = train_ds.class_names
print(class_names)
print(len(class_names)) # 40 classes

In [ ]:
# visualize the train set
# load a batch from train_ds
# load 9 images and show them sequentially in a 3x3 grid

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
    for i in range(7):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.show()

In [ ]:
#normalize the images
from tensorflow.keras.layers import Rescaling

# We normalize the images in our datasets to lie between [0,1] by using a normalization layer
# This layer divides the inputs by 255
# Note that, sometimes the model (if using a pretrained model) expects [-1,1] so we need to adjust our rescaling factor
# Lambda is a shortened function notation that does the equivalent of :
#
#def normalize_batch(x, y):
#    x_norm = normalization_layer(x)
#    return x_norm, y
# map() applies lambda to every batch in the dataset

normalization_layer = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = valid_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

Building the Model

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications import EfficientNetB0


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

# Load EfficientNetB0 without top
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base model for transfer learning
base_model.trainable = False

# Functional API
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs)                 # <-- pass inputs through base_model
x = GlobalAveragePooling2D()(x)
outputs = Dense(40, activation="softmax")(x)

# Create model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile
opt = Adam(learning_rate=0.0005)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()


Training the Model

In [ ]:
# EarlyStopping
stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=10,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)


In [ ]:
# Choice of epochs is arbitrary
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds, # Validation set : used to monitor model performance during training while test is used to evaluate the model after it has been trained
  epochs=epochs,
  callbacks=[checkpoint,stopping]
)

Testing our model

In [ ]:
metrics = model.evaluate(test_ds)
print("Test Loss:", metrics[0])
print("Test Accuracy:", metrics[1])

In [ ]:
images, labels = next(iter(test_ds))  # batch size 32

# Predict **only for this batch**
pred_probs = model.predict(images)
pred_classes = np.argmax(pred_probs, axis=1)

# Number of images to display
num_images = min(len(images), 7)
plt.figure(figsize=(10, 10))

for i in range(num_images):
    ax = plt.subplot(3, 3, i + 1)
    img = images[i]

    # Convert TF tensor to NumPy if needed
    if isinstance(img, tf.Tensor):
        img = img.numpy()

    # Scale to 0-255 if floats
    if img.max() <= 1.0:
        img = (img * 255).astype("uint8")
    else:
        img = img.astype("uint8")

    # Get true and predicted labels
    true_label = class_names[labels[i] if isinstance(labels, np.ndarray) else labels[i].numpy()]
    pred_label = class_names[pred_classes[i]]

    # Color title green if correct, red if wrong
    color = 'green' if true_label == pred_label else 'red'
    plt.title(f"T: {true_label}\nP: {pred_label}", color=color)

    plt.imshow(img)
    plt.axis("off")

plt.tight_layout()
plt.show()